In [5]:
import numpy as np
import pandas as pd
import xgboost as xgb

from datetime import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
from scipy.stats import skew, boxcox
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import itertools

ids = pd.read_csv('../input/test.csv')['id']
train_x = pd.read_csv("../input/train_x_power3.csv")
train_y = pd.read_csv("../input/train_y_power3.csv",header = None)
test_x = pd.read_csv("../input/test_power3.csv")

def fair_obj(preds, dtrain):
    fair_constant = 2
    labels = dtrain.get_label()
    x = (preds - labels)
    den = abs(x) + fair_constant
    grad = fair_constant * x / (den)
    hess = fair_constant * fair_constant / (den * den)
    return grad, hess

shift = 200
def xg_eval_mae(yhat, dtrain):
    y = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(y)-shift,
                                      np.exp(yhat)-shift)



In [6]:
def xgb_model(params , train_x, train_y, ids, test_x ,n_folds = 5,n_print = 500, early_stop = 50,shift=200):
    #n_folds = n_folds
    cv_sum = 0
    fpred = []
    xgb_rounds = []
    test_x = xgb.DMatrix(test_x)
    kf = KFold(train_x.shape[0], n_folds=n_folds,random_state = 2016)

    pred_oob = np.zeros(train_x.shape[0])

    for i, (train_index, test_index) in enumerate(kf):
        print('\n Fold %d' % (i+1))
        X_train, X_val = train_x.iloc[train_index], train_x.iloc[test_index]
        y_train, y_val = train_y.iloc[train_index], train_y.iloc[test_index]

        d_train = xgb.DMatrix(X_train, label=y_train)
        d_valid = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(d_train, 'train'), (d_valid, 'eval')]

        clf = xgb.train(params,
                        d_train,
                        100000,
                        watchlist,
                        early_stopping_rounds=early_stop,
                        obj=fair_obj,
                        verbose_eval = n_print,
                        feval=xg_eval_mae)

        xgb_rounds.append(clf.best_iteration)
        scores_val = clf.predict(d_valid, ntree_limit=clf.best_ntree_limit)
        pred_oob[test_index] = np.exp(scores_val) - shift
        cv_score = mean_absolute_error(np.exp(y_val), np.exp(scores_val))
        print('eval-MAE: %.6f' % cv_score)
        y_pred = np.exp(clf.predict(test_x, ntree_limit=clf.best_ntree_limit)) - shift

        if i > 0:
            fpred = pred + y_pred
        else:
            fpred = y_pred
        pred = fpred
        cv_sum = cv_sum + cv_score

        partial_evalutaion = open('temp_scores_power3.txt','a') 
        partial_evalutaion.write('Fold '+ str(i) + '- MAE:'+ str(cv_score)+'\n')
        partial_evalutaion.flush()


    mpred = pred / n_folds
    score = cv_sum / n_folds
    print('Average eval-MAE: %.6f' % score)
    n_rounds = int(np.mean(xgb_rounds))

    print("Writing results")
    result = pd.DataFrame(mpred, columns=['loss'])
    result["id"] = ids
    result = result.set_index("id")
    print("%d-fold average prediction:" % n_folds)


    now = datetime.now()
    score = str(round((cv_sum / n_folds), 6))
    sub_file = 'test_power3_xgb_fairobj_' + str(score) + '_' + str(
        now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print("Writing submission: %s" % sub_file)
    result.to_csv(sub_file, index=True, index_label='id')

    print("writing out of bag results")
    oob_df = pd.DataFrame(pred_oob, columns = ['loss'])
    sub_file = 'oob_power3_xgb_fairobj_' + str(score) + '_' + str(
        now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print("Writing submission: %s" % sub_file)
    oob_df.to_csv(sub_file, index = False)
    
    return (params,score,result,n_rounds,oob_df)

In [ ]:
params = {
    'seed': 2016,
    'colsample_bytree': 0.35,
    'silent': 1,
    'subsample': 0.8,
    'learning_rate': 0.006,
    'objective': 'reg:linear',
    'max_depth': 9,
    'min_child_weight': 124,
    'nthread' : 20,
    'alpha' : 7.75,
    'gamma':0.9,
    'booster': 'gbtree'}
a1,b1,c1,d1,e1=xgb_model(params , train_x, train_y, ids, test_x,n_folds = 10,n_print = 2500,early_stop = 300)

In [ ]:
params = {
    'seed': 1337,
    'colsample_bytree': 0.35,
    'silent': 1,
    'subsample': 0.8,
    'learning_rate': 0.006,
    'objective': 'reg:linear',
    'max_depth': 9,
    'min_child_weight': 121,
    'nthread' : 20,
    'alpha' : 8,
    'gamma':0.9,
    'booster': 'gbtree'}
a2,b2,c2,d2,e2=xgb_model(params , train_x, train_y, ids, test_x,n_folds = 10,n_print = 2500,early_stop = 300)

In [ ]:
params = {
    'seed': 0,
    'colsample_bytree': 0.45,
    'silent': 1,
    'subsample': 0.8,
    'learning_rate': 0.006,
    'objective': 'reg:linear',
    'max_depth': 10,
    'min_child_weight': 99,
    'nthread' : 20,
    'alpha' : 8.75,
    'gamma':0.8,
    'booster': 'gbtree'}
a3,b3,c3,d3,e3=xgb_model(params , train_x, train_y, ids, test_x,n_folds = 10,n_print = 2500,early_stop = 300)

In [ ]:
params = {
    'seed': 2016,
    'colsample_bytree': 0.35,
    'silent': 1,
    'subsample': 0.75,
    'learning_rate': 0.006,
    'objective': 'reg:linear',
    'max_depth': 12,
    'min_child_weight': 122,
    'nthread' : 18,
    'alpha' : 7.75,
    'gamma':0.85,
    'booster': 'gbtree'}

a4,b4,c4,d4,e4=xgb_model(params , train_x, train_y, ids, test_x,n_folds = 10,n_print = 2500,early_stop = 300)

In [19]:
print b1,b2

1124.879544 1125.110344


In [20]:
print d1,d2

14956 15534
